In [1]:
from IPython.display import clear_output
import numpy as np

from desdeo.emo.methods.bases import baseEA1
from desdeo.emo.operators.crossover import SimulatedBinaryCrossover
from desdeo.emo.operators.mutation import BoundedPolynomialMutation
from desdeo.emo.operators.selection import RVEASelector, NSGAIII_select
from desdeo.emo.operators.generator import RandomGenerator, LHSGenerator
from desdeo.emo.operators.termination import MaxGenerationsTerminator
from desdeo.emo.operators.evaluator import BaseEvaluator
from desdeo.tools.patterns import Publisher, Subscriber
from desdeo.tools.message import SelectorMessageTopics, TerminatorMessageTopics, Message

from desdeo.problem.testproblems import zdt1, binh_and_korn, dtlz2
from desdeo.problem import PolarsEvaluator
import polars as pd

import plotly.express as px

/Users/bhupindersaini/Projects/DESDEO/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
class Printer(Subscriber):
    def __init__(self, publisher) -> None:
        self.print_id = 0
        super().__init__(publisher=publisher, interested_topics=[TerminatorMessageTopics.GENERATION])

    def update(self, message: Message) -> None:
        """Update self as a result of changes in the subject.

        Args:
            msg (dict): the msg from the subject.
        """
        # clear_output(wait=True)
        if message.value % 10 == 0:
            print(f"Generation: {message.value}")

    def state(self) -> dict:
        """Return the state of the subject. This is the msg to be sent to the subscribers."""
        return {}


class visualizer(Subscriber):
    def __init__(self, problem, publisher):
        super().__init__(
            publisher,
            interested_topics=[
                SelectorMessageTopics.SELECTED_TARGETS,
                SelectorMessageTopics.REFERENCE_VECTORS,
                TerminatorMessageTopics.GENERATION,
            ],
        )
        self.problem = problem
        self.fig = {}
        self.vec_fig = {}
        self.current_generation = 0
        self.objective_vectors = None
        self.ideal = list(problem.get_ideal_point().values())
        self.nadir = list(problem.get_nadir_point().values())

    def state(self) -> dict:
        return {}

    def update(self, message: Message) -> None:
        # print(message)
        if message.topic == TerminatorMessageTopics.GENERATION:
            self.current_generation = message.value
        elif message.topic == SelectorMessageTopics.SELECTED_TARGETS:
            self.objective_vectors = np.array(message.value)
            if self.objective_vectors.shape[1] == 2:
                self.fig[self.current_generation] = px.scatter(
                    x=self.objective_vectors[:, 0],
                    y=self.objective_vectors[:, 1],
                    title=f"Generation {self.current_generation}",
                    range_x=[self.ideal[0], self.nadir[0]],
                    range_y=[self.ideal[1], self.nadir[1]],
                )

            elif self.objective_vectors.shape[1] == 3:
                self.fig[self.current_generation] = px.scatter_3d(
                    x=self.objective_vectors[:, 0],
                    y=self.objective_vectors[:, 1],
                    z=self.objective_vectors[:, 2],
                    title=f"Generation {self.current_generation}",
                    range_x=[self.ideal[0], self.nadir[0]],
                    range_y=[self.ideal[1], self.nadir[1]],
                    range_z=[self.ideal[2], self.nadir[2]],
                )

        elif message.topic == SelectorMessageTopics.REFERENCE_VECTORS:
            reference_vectors = np.array(message.value)
            self.vec_fig[self.current_generation] = px.scatter(
                x=reference_vectors[:, 0],
                y=reference_vectors[:, 1],
                title=f"Generation {self.current_generation}",
            )

In [3]:
publisher = Publisher()
# problem = binh_and_korn()
problem = dtlz2(n_variables=10, n_objectives=3)
evaluator = BaseEvaluator(
    problem=problem,
    publisher=publisher,
)

SEED = 0
generator = RandomGenerator(
    problem=problem,
    evaluator=evaluator,
    publisher=publisher,
    n_points=100,
    seed=SEED,
)
crossover = SimulatedBinaryCrossover(
    problem=problem,
    publisher=publisher,
    seed=SEED,
)
mutation = BoundedPolynomialMutation(
    problem=problem,
    publisher=publisher,
    seed=SEED,
)
RVs = np.random.random((100, 3))
RVs = RVs / np.atleast_2d(np.linalg.norm(RVs, axis=1, ord=1)).T


selector = NSGAIII_select(
    problem=problem,
    reference_vectors=RVs,
    publisher=publisher,
    n_survive=100,  # ideal=np.zeros(2), nadir=np.ones(2)
    verbosity=1,
)

selector2 = RVEASelector(
    problem=problem,
    publisher=publisher,
    reference_vectors=RVs,
    verbosity=1,
)
terminator = MaxGenerationsTerminator(1000, publisher=publisher)

#printer = Printer(publisher)
#visualize = visualizer(problem, publisher)

# publisher.subscribe(printer, "current_generation")
# publisher.subscribe(printer, "max_generations")
components: list[Subscriber] = [evaluator, generator, crossover, mutation, selector2, terminator]#, visualize]
[publisher.auto_subscribe(x) for x in components]
[publisher.register_topics(x.provided_topics, x.__class__.__name__) for x in components]

[None, None, None, None, None, None]

In [4]:
# Make this true or false by setting verbosity of selector to 2 or 1
print(publisher.check_consistency())
publisher.relationship_map()

True


{'GENERATION': [('RVEASelector', ['MaxGenerationsTerminator'])],
 'MAX_GENERATIONS': [('RVEASelector', ['MaxGenerationsTerminator'])],
 'NEW_EVALUATIONS': [('MaxGenerationsTerminator', ['BaseEvaluator']),
  ('MaxGenerationsTerminator', ['RandomGenerator'])]}

In [5]:
publisher.subscribers

{<TerminatorMessageTopics.GENERATION: 'GENERATION'>: [<desdeo.emo.operators.selection.RVEASelector at 0x12eef7950>],
 <TerminatorMessageTopics.MAX_GENERATIONS: 'MAX_GENERATIONS'>: [<desdeo.emo.operators.selection.RVEASelector at 0x12eef7950>],
 <EvaluatorMessageTopics.NEW_EVALUATIONS: 'NEW_EVALUATIONS'>: [<desdeo.emo.operators.termination.MaxGenerationsTerminator at 0x12f37e330>],
 <GeneratorMessageTopics.NEW_EVALUATIONS: 'NEW_EVALUATIONS'>: [<desdeo.emo.operators.termination.MaxGenerationsTerminator at 0x12f37e330>]}

In [6]:
%load_ext snakeviz

In [7]:

a = baseEA1(
    evaluator=evaluator,
    generator=generator,
    selection=selector,
    crossover=crossover,
    mutation=mutation,
    termination=terminator,
)

In [8]:
px.scatter_3d(a.outputs["f_1_min", "f_2_min", "f_3_min"],
              x="f_1_min", y="f_2_min", z="f_3_min")

In [9]:
visualize.fig.keys()

NameError: name 'visualize' is not defined

In [76]:
#i=2
fig = visualize.fig[i]
i += 1
fig.show()

In [ ]:
visualize.fig[-1]

In [ ]:
selector.ideal

array([3.49784275e-18, 0.00000000e+00, 0.00000000e+00])

In [ ]:
i = 0
fig = visualize.vec_fig[i]
i += 1
fig.show()